*All content and data presented in the articles on this platform are sourced from the comprehensive boxset titled “Market Risk Analysis” by Carol Alexander. The author of the articles acknowledges and respects the intellectual property rights and copyrights held by the original author of the boxset. The purpose of sharing this information is solely for educational and informational purposes, and no infringement of intellectual property rights is intended.*

In [3]:
from scipy import stats
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
import statsmodels.api as sm

import pathlib
import sys
utils_path = pathlib.Path().absolute().parent.parent
sys.path.append(utils_path.__str__())
import utils.layout as lay
from utils.functions import PCA

# Applications of Term Structure Factor Models

### Example : Portfolio Immunization

*In the article [Term Structure Factor Models](https://baglinifinance.com/blog/term-structure-factor-models/) we estimated a factor model for the UK bond portfolio that is characterized. The factor model is*

$$
-P\&L_{t} \approx £623.74 x P_{1t} + £1001.11 x P_{2t} + £1006.65 x P_{3t} 
$$

*How much should we add of the 10-year bond so that the new portfolio’s P&L is invariant
to changes in the first principal component, i.e. an almost parallel shift in interest rates?
Having done this, how much should we then add of the 5- and 15-year bonds so that the
new portfolio’s P&L is also invariant to changes in the second principal component, i.e. a
change in slope of the yield curve?*

In [34]:
df = pd.read_excel(r"data\Example II.2.1.xls", sheet_name="PV01").iloc[:, :4]
spot = pd.read_excel(r"data\Example II.2.1.xls", sheet_name="Spot").set_index("Date")
spot_cols = [str(i) + "y" for i in spot.columns]
spot.columns = spot_cols
diff_bps = spot.diff()[1:] *100
df["PV01"] = (df["CashFlow"]*100*df["Maturity"]*(1+df["SpotRate"])**(-(df["Maturity"]+1))).astype(float)

In [35]:
cov_mx = diff_bps.cov()
eigenvals, eigenvecs = PCA(cov_mx)
net_sensitivities = df["PV01"].values.T.dot(eigenvecs)
princ_comps = diff_bps.dot(eigenvecs)
net_sensitivities = df["PV01"].values.T.dot(eigenvecs)


In [36]:
df["PV01"]

0     273.391825
1     352.149464
2     253.417420
3     161.577567
4     385.856772
5     552.735069
6     985.541444
7     269.077847
8      57.882935
9     123.044119
10    518.166803
11    676.632505
12    175.576011
13     72.500888
14   -372.502121
15   -381.262678
16   -777.789769
17   -593.249481
18   -401.171160
19   -202.999032
Name: PV01, dtype: float64

In [32]:
import numpy as np

# Define the two series
series1 = df["PV01"]
series2 = eigenvecs["λ1"] # Replace with your second series

# Define a function to calculate the dot product
def calculate_dot_product(series1, series2):
    return np.dot(series1, series2)

# Define a tolerance level for the dot product to be considered "zero"
tolerance = 1e-6
tolerance1 = -1e-6

# Start an iterative process to adjust values in series1
max_iterations = 1000  # Set a maximum number of iterations to avoid infinite loops
for _ in range(max_iterations):
    dot_product = calculate_dot_product(series1, series2)
    print(dot_product)

    # Check if the dot product is close enough to zero
    if abs(dot_product) < tolerance:
        break  # Exit the loop if the dot product is within tolerance

    # If not, adjust the values in series1
    # For example, you can decrease the first element of series1 by a small step size
    series1[11] -= 4  # Adjust this step size as needed
    print(series1[11])
    

601.0778956226293
572.6325046710556
600.1712900912305
568.6325046710556
599.2646845598317
564.6325046710556
598.3580790284329
560.6325046710556
597.4514734970342
556.6325046710556
596.5448679656354
552.6325046710556
595.6382624342366
548.6325046710556
594.7316569028378
544.6325046710556
593.825051371439
540.6325046710556
592.9184458400402
536.6325046710556
592.0118403086415
532.6325046710556
591.1052347772427
528.6325046710556
590.1986292458439
524.6325046710556
589.2920237144451
520.6325046710556
588.3854181830463
516.6325046710556
587.4788126516476
512.6325046710556
586.5722071202488
508.6325046710556
585.66560158885
504.6325046710556
584.7589960574512
500.6325046710556
583.8523905260524
496.6325046710556
582.9457849946536
492.6325046710556
582.0391794632549
488.6325046710556
581.1325739318561
484.6325046710556
580.2259684004573
480.6325046710556
579.3193628690585
476.6325046710556
578.4127573376597
472.6325046710556
577.5061518062607
468.6325046710556
576.599546274862
464.6325046710

<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: S


505.8843148257566
152.63250467105559
504.9777092943578
148.63250467105559
504.071103762959
144.63250467105559
503.16449823156023
140.63250467105559
502.25789270016145
136.63250467105559
501.35128716876267
132.63250467105559
500.4446816373639
128.63250467105559
499.5380761059651
124.63250467105559
498.6314705745662
120.63250467105559
497.7248650431674
116.63250467105559
496.81825951176864
112.63250467105559
495.91165398036986
108.63250467105559
495.0050484489711
104.63250467105559
494.0984429175723
100.63250467105559
493.1918373861735
96.63250467105559
492.28523185477474
92.63250467105559
491.37862632337595
88.63250467105559
490.4720207919772
84.63250467105559
489.5654152605784
80.63250467105559
488.6588097291796
76.63250467105559
487.7522041977808
72.63250467105559
486.84559866638205
68.63250467105559
485.93899313498326
64.63250467105559
485.0323876035845
60.632504671055585
484.1257820721857
56.632504671055585
483.2191765407869
52.632504671055585
482.312571009388
48.632504671055585
48

<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: S

-247.36749532894441
414.317156154479
-251.36749532894441
413.4105506230802
-255.36749532894441
412.50394509168143
-259.3674953289444
411.59733956028253
-263.3674953289444
410.69073402888375
-267.3674953289444
409.78412849748497
-271.3674953289444
408.8775229660862
-275.3674953289444
407.9709174346874
-279.3674953289444
407.0643119032886
-283.3674953289444
406.15770637188984
-287.3674953289444
405.25110084049106
-291.3674953289444
404.3444953090923
-295.3674953289444
403.4378897776935
-299.3674953289444
402.5312842462947
-303.3674953289444
401.62467871489594
-307.3674953289444
400.71807318349715
-311.3674953289444
399.8114676520984
-315.3674953289444
398.9048621206996
-319.3674953289444
397.9982565893008
-323.3674953289444
397.091651057902
-327.3674953289444
396.18504552650325
-331.3674953289444
395.27843999510435
-335.3674953289444
394.37183446370557
-339.3674953289444
393.4652289323068
-343.3674953289444
392.558623400908
-347.3674953289444
391.6520178695092
-351.3674953289444
390.7454

...

<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: S


-145.96506224997216
-2723.3674953289446
-146.87166778137095
-2727.3674953289446
-147.77827331276973
-2731.3674953289446
-148.6848788441685
-2735.3674953289446
-149.5914843755673
-2739.3674953289446
-150.4980899069662
-2743.3674953289446
-151.40469543836497
-2747.3674953289446
-152.31130096976375
-2751.3674953289446
-153.21790650116253
-2755.3674953289446
-154.1245120325613
-2759.3674953289446
-155.0311175639601
-2763.3674953289446
-155.93772309535888
-2767.3674953289446
-156.84432862675766
-2771.3674953289446
-157.75093415815644
-2775.3674953289446
-158.65753968955522
-2779.3674953289446
-159.564145220954
-2783.3674953289446
-160.47075075235279
-2787.3674953289446
-161.37735628375157
-2791.3674953289446
-162.28396181515035
-2795.3674953289446
-163.19056734654913
-2799.3674953289446
-164.0971728779479
-2803.3674953289446
-165.0037784093467
-2807.3674953289446
-165.91038394074548
-2811.3674953289446
-166.81698947214437
-2815.3674953289446
-167.72359500354315
-2819.3674953289446
-168.630

<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: S

-2987.3674953289446
-206.70763285369105
-2991.3674953289446
-207.61423838508983
-2995.3674953289446
-208.52084391648862
-2999.3674953289446
-209.4274494478874
-3003.3674953289446
-210.33405497928618
-3007.3674953289446
-211.24066051068496
-3011.3674953289446
-212.14726604208374
-3015.3674953289446
-213.05387157348252
-3019.3674953289446
-213.9604771048813
-3023.3674953289446
-214.8670826362801
-3027.3674953289446
-215.77368816767887
-3031.3674953289446
-216.68029369907765
-3035.3674953289446
-217.58689923047643
-3039.3674953289446
-218.49350476187522
-3043.3674953289446
-219.400110293274
-3047.3674953289446
-220.30671582467278
-3051.3674953289446
-221.21332135607156
-3055.3674953289446
-222.11992688747034
-3059.3674953289446
-223.02653241886912
-3063.3674953289446
-223.9331379502679
-3067.3674953289446
-224.8397434816667
-3071.3674953289446
-225.74634901306547
-3075.3674953289446
-226.65295454446425
-3079.3674953289446
-227.55956007586303
-3083.3674953289446
-228.46616560726181
-3087.3

<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series1[11] -= 4  # Adjust this step size as needed
<ipython-input-32-34e37b9d7dbd>:27: S

In [21]:
# Print the optimized series1
print(f'Optimized Series 1: {series1}')

# Verify that the dot product is close to zero
final_dot_product = calculate_dot_product(series1, series2)
print(f'Final Dot Product: {final_dot_product}')

Optimized Series 1: 0     2.733918e+02
1     3.521495e+02
2     2.534174e+02
3     1.615776e+02
4     3.858568e+02
5     5.527351e+02
6     9.855414e+02
7     2.690778e+02
8     5.788293e+01
9     1.230441e+02
10    5.181668e+02
11   -9.999323e+06
12    1.755760e+02
13    7.250089e+01
14   -3.725021e+02
15   -3.812627e+02
16   -7.777898e+02
17   -5.932495e+02
18   -4.011712e+02
19   -2.029990e+02
Name: PV01, dtype: float64
Final Dot Product: -2265890.085463063


In [18]:
series1[11]

-9999323.367495328

In [7]:
from scipy.optimize import root

def equation(vars):
    a, b = vars
    return  a.values.T.dot(b)

initial_guess = [ df["PV01"], eigenvecs["λ1"]]
result = root(equation, initial_guess)

optimal_solution = result.x
print(f'Optimal Solution: {optimal_solution}')

ValueError: too many values to unpack (expected 2)